In [ ]:
import time

import pandas as pd
from IPython.display import display
from cmajiang import Shoupai

In [ ]:
df = pd.read_csv('question.csv')
questions = df.to_dict('records')

In [ ]:
for q in questions[:3]:
  print('======================================')
  print(f'場風: {q["round_wind"]}, 自風: {q["player_wind"]}')
  print('ドラ表示牌')
  display(Shoupai(q['dora_str']))
  print('------------------------------------------------------------------')
  print('手牌')
  display(Shoupai(q['hand']))
  print('リーチ' if q['is_riichi'] else '', 'ツモ' if q['is_tsumo'] else 'ロン', '海底摸月' if q['is_haitei'] else '', '河底撈魚' if q['is_houtei'] else '')
  display(Shoupai(q['win_tile']))

  time.sleep(1)
  answer = input('回答: ')
  answer_split = answer.split(' ')

  correct = False
  if len(answer_split) == 1:
    if answer == str(q["hand_value_cost_main"]) and q["hand_value_cost_additional"] == 0:
      correct = True
  elif len(answer_split) == 2:
    if answer == f'{q["hand_value_cost_additional"]} {q["hand_value_cost_main"]}':
      correct = True
  else:
    print('入力形式が間違っています')

  if correct:
      print('正解です')
  else:
    print(f'間違っています。正解は {q["hand_value_han"]}飜{q["hand_value_hu"]}符, {q["hand_value_cost_additional"]}, {q["hand_value_cost_main"]}です。')
    print(f'役: {q["hand_value_yaku"]}')

print("お疲れ様でした。全ての問題が終了しました")
